### Домашняя работа к уроку № 4:
#### Парсинг HTML. XPath

Выберите веб-сайт с табличными данными, который вас интересует.

Напишите код Python, использующий библиотеку requests для отправки HTTP GET-запроса на сайт и получения HTML-содержимого страницы.

Выполните парсинг содержимого HTML с помощью библиотеки lxml, чтобы извлечь данные из таблицы.

Сохраните извлеченные данные в CSV-файл с помощью модуля csv.

Ваш код должен включать следующее:

Строку агента пользователя в заголовке HTTP-запроса, чтобы имитировать веб-браузер и избежать блокировки сервером.
Выражения XPath для выбора элементов данных таблицы и извлечения их содержимого.
Обработка ошибок для случаев, когда данные не имеют ожидаемого формата.
Комментарии для объяснения цели и логики кода.

In [17]:
import requests
from lxml import html
import csv
import pandas as pd
from fake_useragent import UserAgent ### Для указания агента пользователя
import re
from urllib.parse import urljoin

In [2]:
### Предлагаю для парсинга выбрать таблицу из русской Википедии со статистикой численности населения городов России с населением более 100 тысяч жителей.
### Первые два столбца с рангами городов предлагаю не сохранять, т.к. в них нет необходимости, т.к. рейтинг можно вычислить на основании данных распарсиных из этой таблицы.

url_base = "https://ru.wikipedia.org"
url = "https://ru.wikipedia.org/wiki/Список_городов_России_с_населением_более_100_тысяч_жителей"

In [3]:
ua = UserAgent() #### для указания "браузера"
headers = {
    "User-Agent": ua.firefox, ### генерируем данные браузера Мозила
}

In [4]:
# Отправка HTTP GET запроса на целевой URL с пользовательским заголовком User-Agent
try:
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        print("Успешный запрос API по URL: ", response.url)
    else:
        print("Запрос API отклонен с кодом состояния:", response.status_code)

except requests.exceptions.RequestException as e:
    print("Ошибка в осущественнии HTML запроса:", e)

Успешный запрос API по URL:  https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8_%D1%81_%D0%BD%D0%B0%D1%81%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5%D0%BC_%D0%B1%D0%BE%D0%BB%D0%B5%D0%B5_100_%D1%82%D1%8B%D1%81%D1%8F%D1%87_%D0%B6%D0%B8%D1%82%D0%B5%D0%BB%D0%B5%D0%B9


In [5]:
# Парсинг HTML-содержимого ответа с помощью библиотеки lxml с обработкой исключительных ситуаций
try:
    tree = html.fromstring(response.content)
    print(tree)
except html.etree.ParserError as e:
    print("Ошибка в парсинге HTML содержимого:", e)


<Element html at 0x2119739ee90>


In [6]:
try:
    table_rows = tree.xpath("//table[@class='wikitable sortable']/tbody/tr")
    len(table_rows)
    
except IndexError as e:
    print("Ошибка доступа к результату:", e)

except Exception as e:
    print("Произошла непредвиденная ошибка:", e)

In [51]:
### получаем заголовки столбцов таблицы
try:
    years = table_rows[0].xpath("//th/span/text()")[2:]
    print(years)
    
except IndexError as e:
    print("Ошибка доступа к результату:", e)

except Exception as e:
    print("Произошла непредвиденная ошибка:", e)

['1897', '1926', '1939', '1959', '1970', '1979', '1989', '2002', '2010', '2016', '2021', '2022', '2023', '2024']


In [52]:
### Добавляем два столбца в заголовок таблицы - название города и ссылку на другую страницу в Википедии, где описана подробная информациия о городе
years.insert(0, "Город")
years.append("Ссылка на информацию о городе")

### Создаем заготовку для таблицы (пустую таблицу) с названиями столбцов
df = pd.DataFrame(columns = years)
print(years)


['Город', '1897', '1926', '1939', '1959', '1970', '1979', '1989', '2002', '2010', '2016', '2021', '2022', '2023', '2024', 'Ссылка на информацию о городе']


In [57]:
### Собственно парсинг таблицы
try:
    for row in table_rows[2:]:
        ### Работа с каждой ячейкой таблицы отдельно
        cells = row.xpath('.//td|.//th')[2:]
        
        ### Если в ячейке таблицы значения нет, то возвращаем None 
        row_data = [cell.text_content().strip() if cell.text_content().strip() else None for cell in cells]
        
        ### Если в ячейке таблицы прочерк, то замещаем его на None
        row_data = [None if item == '—' else item for item in row_data]
        
        ### убираем референсные ссылки на источники статистических данных
        row_data = [re.sub("\[[0-9]+\]", '', s) if s is not None else None for s in row_data]
        
        ### получаем ссылку на информацию о городе
        city_wiki_ref = urljoin(url_base, row.xpath(".//td//a/@href")[0])
        
        ### Объединяем данные из строки в итоговую таблицу
        temp_df = pd.DataFrame(data = row_data)
        temp_df = temp_df.transpose()
        temp_df.columns = years[:-1]
        temp_df["Ссылка на информацию о городе"] = city_wiki_ref
        df = pd.concat([df, temp_df])
        #print(temp_df)
    
except IndexError as e:
    print("Ошибка доступа к результату:", e)

except Exception as e:
    print("Произошла непредвиденная ошибка:", e)

In [58]:
### Смотрим на готовую итоговую таблицу
df

,Город,1897,1926,1939,1959,1970,1979,1989,2002,2010,2016,2021,2022,2023,2024,Ссылка на информацию о городе
0,Москва,1039,2080,4609,6133,7194,8057,8878,10126,11504,12330,13010,13015,13104,13150,https://ru.wikipedia.org/wiki/%D0%9C%D0%BE%D1%...
0,Санкт-Петербург,1265,1737,3431,3390,4033,4569,4989,4661,4880,5226,5602,5608,5600,5598,https://ru.wikipedia.org/wiki/%D0%A1%D0%B0%D0%...
0,Новосибирск,8,120,404,885,1161,1309,1420,1426,1474,1584,1634,1636,1635,1634,https://ru.wikipedia.org/wiki/%D0%9D%D0%BE%D0%...
0,Екатеринбург,43,140,423,779,1025,1210,1296,1294,1350,1444,1544,1543,1539,1536,https://ru.wikipedia.org/wiki/%D0%95%D0%BA%D0%...
0,Казань,130,179,406,667,869,989,1085,1105,1144,1217,1309,1312,1315,None,https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D0%...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Бердск,None,5,11,29,53,67,79,89,97,103,103,None,103,None,https://ru.wikipedia.org/wiki/%D0%91%D0%B5%D1%...
0,Элиста,None,None,17,23,50,70,90,104,104,104,103,None,102,None,https://ru.wikipedia.org/wiki/%D0%AD%D0%BB%D0%...
0,Ногинск,11,39,81,95,104,119,123,118,100,102,104,None,102,None,https://ru.wikipedia.org/wiki/%D0%9D%D0%BE%D0%...
0,Новошахтинск,None,7,48,104,102,103,108,101,111,109,103,103,102,None,https://ru.wikipedia.org/wiki/%D0%9D%D0%BE%D0%...


In [59]:
# Сохранение DataFrame в файл CSV
df.to_csv('List of Russian cities population.csv', index=False)